In [1]:
conda install -c conda-forge python-dotenv


Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [2]:
# get api key from your .env file

import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

7h3hkWvV4KEqzoZrEJmF


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key="+API_KEY
r = requests.get(url)


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

print(r.json())

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [6]:
##1

url17='https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY

r17 = requests.get(url17)

In [19]:
##2 

json_data17 = r17.json()

#print(json_data17)
type(json_data17)

dict

In [26]:
##3

#The opening price is index 1 of each row. Want to find highest and lowest opening prices
open_list = []
for entry in json_data17['dataset_data']['data']:
    if entry[1] != None :
        open_list.append(entry[1])
#print(open_list)    
top = str(max(open_list))

low = str(min(open_list))

print('The highest value was: $' + top + ' and the lowest was: $' + low + '.')

The highest value was: $53.11 and the lowest was: $34.0.


In [28]:
##4

# Largest change in any one day. "High" is index 2; "Low" is index 3.

change_list = []

for entry in json_data17['dataset_data']['data']:
    if (entry[2:3] != None):
        change_list.append(entry[2]-entry[3])
    else:
        pass
    
print('The biggest change in any one day was: $' + str(round(max(change_list),2)) + '.')

The biggest change in any one day was: $2.81.


In [51]:
##5

#What was the largest change between any two days (based on Closing Price)?

close_list = []

for entry in json_data17['dataset_data']['data']:
    close_list.append(entry[4])

#print(close_list) #there are no Nones

length=len(close_list)
#print(length)

diffs=[]
perc_changes=[]

for i in range(length-1):
    diffs.append(close_list[i+1]-close_list[i]) #starts with day 1 minus day 0
    
    
for i in range(length-1):
    perc_changes.append(diffs[i]/close_list[i]) #starts with day 1 minus day 0 over day 0
    
mx=str(round(max(perc_changes),4))
mn=str(round(min(perc_changes),4))

print(mx)
print(mn)

print('The largest two day change was 6.12% increase.')



0.0612
-0.0384
The largest two day change was 6% increase.


In [53]:
##6 

#What was the average daily trading volume during this year? Index 6

vols=[]

for entry in json_data17['dataset_data']['data']:
    if entry[6] != None:
        vols.append(entry[6])


avg_vol = round(sum(vols)/len(vols),2)

print('The average trading volume was ' + str(avg_vol))

The average trading volume was 89124.34


In [55]:
##7 

#Median trading volume

sorted_vol = sorted(vols)

median = sorted_vol[(len(sorted_vol)//2)]
print('Median Trading volume is ' + str(median))

Median Trading volume is 76286.0
